# **Лабораторна робота №1. Системи DS**
# Система прогнозування цін будинків в окрузі Кінг
# КМ-81. Горбач Костянтин
# Розуміння бізнесу — розуміння даних. Підготовка даних для вирішення конкретної задачі.

# **1.1 Розуміння бізнесу - розуміння даних**

Визначимося з предметною областю. Будемо досліждувати вторинний ринок нерухомості (а саме житлові будинки) в окрузі Кінг. Метою цього дослідження є підготовка та обробка даних для побудови моделей прогнозування цін будинків. Основною задачею є прогнозування ціни будинку в залежності від множини параметрів (площа, місцезнаходження, стан будинку, розмір гаражу тощо). 

Підготовлені та оброблені дані з данної лабораторної роботи будуть використані в другій лабраторній роботі для створення моделі прогнозування ціни. Ця модель буде корисна для рієлторів. Наприклад, рієлтор, знаючи параметри будинку, зможе отримати оптимальну ціну, за якою цей будинок можна продати, або зможе отримати приблизну ринкову вартість будинку, що ще не виставленний на продаж.

# Збір данних з інтернет-ресурсів

Дані були взяті на сайті kaggle: https://www.kaggle.com/harlfoxem/housesalesprediction.

Імпорт модулів

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

Шлях до даних:

In [ ]:
data_path = '/kaggle/input/housesalesprediction/kc_house_data.csv'

# Збереження та опис даних

Збережемо дані з .csv в dataframe з pandas для спрощення роботи з ними:

In [ ]:
df_data = pd.read_csv(data_path)

Виведемо таблицю з даними:

In [ ]:
df_data

Виведемо інформацію про колонки та розмір таблиці:

In [ ]:
df_data.info()

In [ ]:
df_data.shape

Опишемо основні з характеристик:

* sqft_living (int64) — житлова площа.
* yr_built (int64) — рік побудови.
* floors (int64) — кількість поверхів.
* bedrooms (int64) — кількість спальних кімнат.
* bathrooms (int64) — кількість вбиралень.
* price (int64) — ціна продажу нерухомості в доларах. Змінна, яку необхыдно прогнозувати.
тд.

# 1.2 Підготовка даних

# Первинна очистка даних

Проведемо очистку данних. На перший погляд, датасет немає проблем з втраченими даними. Перевіримо це та почнемо аналіз даних.

Приберемо id та дату, які не потрібні нам

In [ ]:
df_data.drop('id', axis = 1, inplace = True)
df_data.drop('date', axis = 1, inplace = True)

In [ ]:
df_data

Оцінимо цілісність даних

In [ ]:
data_df_nans = 100 * df_data.isnull().sum() / len(df_data)
data_df_nans = data_df_nans.drop(data_df_nans[data_df_nans == 0].index).sort_values(ascending = False)
missing_data_ratio = pd.DataFrame({'Втрачені дані (в %)': data_df_nans})
missing_data_ratio

Втрат даних немає, тому можемо приступити до аналізу.

# Розвідувальний аналіз

Після того, як ми оцінили цілісність даних і виправили помилки в цілісності (в нашому випадку, таких не було виявлено), приступимо до розвідувального аналізу.

Подивимося, як скорельовані відносно один одного дані

In [ ]:
corr_matrix = df_data.corr()
plt.subplots(figsize=(16, 12))
sns.heatmap(corr_matrix, vmin = -1, vmax = 1, annot = True, square=True)

Приберемо колонки, кореляція з ціною яких менша по модулю за 10% для того, щоб залишити колонки, які мають більший вплив на ціну.

In [ ]:
not_corr_colls = ('sqft_lot', 'condition', 'yr_built', 'yr_renovated', 'zipcode', 'long', 'sqft_lot15')
for col in not_corr_colls:
    df_data.drop(col, axis = 1, inplace = True)

In [ ]:
df_data

Видно, що дані з самого початку були очищенні і досить гарно підготовані до обробки. Оцінимо основні статистичні характеристики:

In [ ]:
df_data.describe()

Було отримано, що в середньому, будинки які ми аналізуємо в датасеті є одноповерховими та з трьома спальними кімнатами, тобто є будинками низького-середнього цінового сегменту.

Побудуємо графік попарної залежності даних (виведемо половину даних, для пришвидшення роботи).

In [ ]:
sns.pairplot(df_data.sample(10000))

З кореляційної матриці (яка була отримана вище) та графіка можна сказати, що найбільший позитивний вплив (чим більше показник, тим більше ціна) на ціну має житлова площа та кількість спальних кімнат.

Виділимо в окремий массив значення, яке будемо прогнозувати - ціну (price)

In [ ]:
target_vals = df_data.price.values
target_vals

In [ ]:
df_data_final = df_data.drop('price', axis = 1)

# Розбиття набору даних

Розділимо дані на тренувальний та тестовий датасет (70% / 30%). Для оцінки середньоквадратичного значення prediction моделі, нормалізуємо target значенння.

In [ ]:
n_data_length = df_data.shape[0]
train_data_shape = int(n_data_length * 0.7)
df_train = df_data_final[:train_data_shape]
df_test = df_data_final[train_data_shape:]
train_target = target_vals[:train_data_shape]
test_target = target_vals[train_data_shape:]

train_target_avg, test_target_avg = np.mean(train_target), np.mean(test_target)
train_target_std, test_target_std = np.std(train_target), np.std(test_target)
train_target = (train_target - train_target_avg) / train_target_std
test_target = (test_target - test_target_avg) / test_target_std

# Побудування моделі (опціонально)

Для перевірки того, чи підходять дані для майбутнього використання, побудуємо на їх базі просту лінійну модель та виведемо показник RMSE.

In [ ]:
lm = LinearRegression()
lm.fit(df_train, train_target)
prediction = lm.predict(df_test)
print(np.sqrt(mean_absolute_error(test_target, prediction)))

Як приклад, покажемо залежність розміру будівлі та її ціни: 

In [ ]:
plt.scatter(df_data["sqft_living"], df_data['price'], color = 'green')
sns.regplot(x = df_data["sqft_living"], y = df_data['price'], data = df_data, scatter = False)
plt.xlabel("Square meters living")
plt.ylabel("Price")
plt.show()

# Бізнес-профіль Еріксона-Пенкера

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/input/diagramlab1/diagram-ds-lab1 (1).png') 

# 1.3 Верифікація та валідація результатів

# Верифікація

В даній лабораторній роботі була проведена підготовка даних для вирішення конкретної задачі, прогнозування цін будинків. Необхідно було поставити задачу, визначивши предметну область, та дослідити дані, що були знайдені на сайті kaggle.com.


* Предметна область - вторинний ринок нерухомості в King County.
* Задача - підготовка даних до побудови системи прогнозування ціни будинку в залежності від характеристик будинку (житлова площа, кількість кімнат, кількість поверхів і тд).

Дані було завантажено та збережено в Dataframe, також, була описана структура таблиці, її розмірність та обчислені основні статистичні характеристики даних в таблиці.

В процесі написання роботи та обробки даних було проведено:
* Усунення NaN значень (не було знайдено)
* Усунення несуттєвих або некорельованих значень
* Обраховані основні статистичні характеристики
* Датасет розбито на тренувальну та тестову вибірку

Задокументовано класи бізнес-профілю Еріксона-Пенкера, виконано усі вище зазначені етапи, тобто система відповідає умовам, а отже, верифікація пройшла успішно.

# Валідація

Після обробки датасету, можна сказати, що дані не мають дефектів, є цілісними і коректними для даної задач. Проведено аналіз ознак та впливу їх один на одного та на ціну будинку. В даному датасеті майже всі дані мали слабкий вплив на ціну будинку. Також, було виявлено, які показники більше всьго впливають на ціну будинку: "bedrooms", "sqft_basement", "lat". Але більшість показників мають коефіцієнт кореляції по модулю менший за 0.5, тому можна сказати, що прогноз моделі може бути не точним при додаванні більших за впливом характеристик.

# 1.4 Висновки

В даній лабораторній роботі була проведена підготовка даних для вирішення конкретної задачі, прогнозування цін будинків для округу Кінг. В процесі підготовки даних був використаний Python (numpy, pandas, sklearn, matplotlib, seaborn). 

Основні відомості про задачу:

* Предметна область - ринок нерухомості в King County.
* Задача - підготовка даних до побудови системи прогнозування ціни будинку в залежності від цих характеристик.

Було пройдено основні етапи створення систем DS:
* Збір даних
* Обробка даних
* Видалення не потрібних для аналізу даних
* Аналіз даних
* Розбиття на тренувальні та тестові набори
* Верифікація та валідація результатів
Дані було завантажено та збережено в Dataframe, також, була описана структура таблиці, її розмірність та обчислені основні статистичні характеристики даних в таблиці.

Можна зробити висновок, що дані не мають дефектів, є цілісними і коректними для даної задач. Проведено аналіз ознак та впливу їх один на одного та на ціну будинку. В даному датасеті майже всі дані мали слабкий вплив на ціну будинку. Також, було виявлено, які показники більше всьго впливають на ціну будинку: "bedrooms", "sqft_basement", "lat". Але більшість показників мають коефіцієнт кореляції по модулю менший за 0.5, тому можна сказати, що прогноз моделі може бути не точним при додаванні більших за впливом характеристик.
В результаті дані були підготовані до наступного етапу розробки - застосування алгоритмів для прогнозування цін. Для перевірки, була побудована проста модель. Перевірка пройшла без проблем (не дивлячись на середню точність моделі), а значить, данні готові до використання.